In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import warnings
warnings.filterwarnings('ignore')

# Setze Random Seeds
np.random.seed(42)
torch.manual_seed(42)

# Geräteerkennung für Mac GPU (MPS)
if torch.backends.mps.is_available():
    device = torch.device('mps')
    print(f"Using device: MPS (Apple Silicon GPU)")
elif torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"Using device: CUDA GPU")
else:
    device = torch.device('cpu')
    print(f"Using device: CPU")

print(f"PyTorch Version: {torch.__version__}")

Using device: MPS (Apple Silicon GPU)
PyTorch Version: 2.9.1


In [2]:
train_path = Path('../train.csv')
test_path = Path('../test.csv')

print("Lade Daten...")
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

print(f"Train Shape: {train_df.shape}")
print(f"Test Shape: {test_df.shape}")
print("\nErste Zeilen Train:")
print(train_df.head())
print("\nSpalten:")
print(train_df.columns.tolist())
print("\nDatentypen:")
print(train_df.dtypes)
print("\nFehlende Werte:")
print(train_df.isnull().sum())

Lade Daten...
Train Shape: (16512, 10)
Test Shape: (4128, 10)

Erste Zeilen Train:
   longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0    -117.03     32.71                33.0       3126.0           627.0   
1    -118.16     33.77                49.0       3382.0           787.0   
2    -120.48     34.66                 4.0       1897.0           331.0   
3    -117.11     32.69                36.0       1421.0           367.0   
4    -119.80     36.78                43.0       2382.0           431.0   

   population  households  median_income  median_house_value ocean_proximity  
0      2300.0       623.0         3.2596            103000.0      NEAR OCEAN  
1      1314.0       756.0         3.8125            382100.0      NEAR OCEAN  
2       915.0       336.0         4.1563            172600.0      NEAR OCEAN  
3      1418.0       355.0         1.9425             93400.0      NEAR OCEAN  
4       874.0       380.0         3.5542             96500.0          I

In [3]:
# Target separieren
target_col = 'median_house_value'
X_train_full = train_df.drop(columns=[target_col])
y_train_full = train_df[target_col].values

# ============================================
# 🚀 LÖSUNG 1: QUANTIL-BASIERTE AUSREISSER-ENTFERNUNG
# WICHTIG: Nur untere Ausreißer entfernen, ALLE teuren Häuser behalten!
# ============================================
print("=" * 60)
print("QUANTIL-BASIERTE AUSREISSER-ENTFERNUNG")
print("=" * 60)

# Nur unterste 1% entfernen (sehr billige Fehldaten)
# KEINE obere Grenze! → Verhindert Ceiling Effect
lower_quantile = np.percentile(y_train_full, 1)  # Nur unterste 1%

print(f"Original Train-Größe: {len(y_train_full)}")
print(f"Untere Grenze (1% Quantil): ${lower_quantile:,.0f}")
print(f"Max Preis im Dataset: ${y_train_full.max():,.0f}")
print(f"Preisverteilung:")
print(f"  - 25% Quantil: ${np.percentile(y_train_full, 25):,.0f}")
print(f"  - 50% Quantil: ${np.percentile(y_train_full, 50):,.0f}")
print(f"  - 75% Quantil: ${np.percentile(y_train_full, 75):,.0f}")
print(f"  - 99% Quantil: ${np.percentile(y_train_full, 99):,.0f}")

# Entferne nur untere Ausreißer
outlier_mask = y_train_full >= lower_quantile  # KEINE obere Grenze!
n_outliers = (~outlier_mask).sum()

print(f"\nEntfernte Ausreißer: {n_outliers} ({n_outliers/len(y_train_full)*100:.2f}%)")

# Entferne Ausreißer
X_train_full = X_train_full[outlier_mask]
y_train_full = y_train_full[outlier_mask]

print(f"Neue Train-Größe: {len(X_train_full)}")
print(f"Neuer Max Preis: ${y_train_full.max():,.0f}")
print("✅ Alle teuren Häuser bleiben erhalten (kein Ceiling!)")

# ============================================
# 🔥 FIX BUG 1: LOG-TRANSFORM VOR FEATURE ENGINEERING!
# ============================================
print("\n" + "=" * 60)
print("LOG-TRANSFORM (VOR FEATURE ENGINEERING!)")
print("=" * 60)

# LOG-TRANSFORM ZUERST anwenden!
y_train_full_log = np.log1p(y_train_full)
print(f"✅ Log-Transform angewendet BEVOR Feature Engineering")
print(f"   Original Max: ${y_train_full.max():,.0f}")
print(f"   Log Max: {y_train_full_log.max():.2f}")

# ============================================
# 🚀 LÖSUNG 2: FEATURE ENGINEERING
# ============================================
print("\n" + "=" * 60)
print("FEATURE ENGINEERING")
print("=" * 60)

# Neue Features erstellen (OHNE price_per_room!)
# price_per_room entfernt, da Test-Daten keinen Target haben!

# 1. Rooms per household (Wohnungsgröße-Indikator)
X_train_full['rooms_per_household'] = X_train_full['total_rooms'] / (X_train_full['households'] + 1)

# 2. Bedrooms ratio (Wohnungstyp-Indikator)
X_train_full['bedrooms_ratio'] = X_train_full['total_bedrooms'] / (X_train_full['total_rooms'] + 1)

# 3. Population density (Urban vs Rural)
X_train_full['population_density'] = X_train_full['population'] / (X_train_full['households'] + 1)

# 4. Income per bedroom (Wohlstands-Indikator)
X_train_full['income_per_bedroom'] = X_train_full['median_income'] / (X_train_full['total_bedrooms'] + 1)

print(f"✅ 4 neue Features erstellt:")
print("  - rooms_per_household")
print("  - bedrooms_ratio")
print("  - population_density")
print("  - income_per_bedroom")
print("\n❌ price_per_room ENTFERNT (Test-Daten haben keinen Target!)")

# Update y_train_full zu log-transformierter Version
y_train_full = y_train_full_log

# IDs speichern falls vorhanden
id_cols = ['id'] if 'id' in X_train_full.columns else []
test_ids = test_df[id_cols].copy() if id_cols else None

# IDs entfernen
X_train_full = X_train_full.drop(columns=id_cols, errors='ignore')
X_test = test_df.drop(columns=id_cols, errors='ignore')

# Feature Engineering auch auf Test-Daten (OHNE price_per_room!)
X_test['rooms_per_household'] = X_test['total_rooms'] / (X_test['households'] + 1)
X_test['bedrooms_ratio'] = X_test['total_bedrooms'] / (X_test['total_rooms'] + 1)
X_test['population_density'] = X_test['population'] / (X_test['households'] + 1)
X_test['income_per_bedroom'] = X_test['median_income'] / (X_test['total_bedrooms'] + 1)

# Categorical Features (ocean_proximity)
categorical_cols = X_train_full.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X_train_full.select_dtypes(include=[np.number]).columns.tolist()

print(f"\nCategorical Features: {categorical_cols}")
print(f"Numerical Features: {len(numerical_cols)}")

# One-Hot Encoding für kategorische Features
if categorical_cols:
    X_train_full = pd.get_dummies(X_train_full, columns=categorical_cols, drop_first=True)
    X_test = pd.get_dummies(X_test, columns=categorical_cols, drop_first=True)
    
    # Stelle sicher, dass Train und Test die gleichen Spalten haben
    missing_cols = set(X_train_full.columns) - set(X_test.columns)
    for col in missing_cols:
        X_test[col] = 0
    X_test = X_test[X_train_full.columns]

print(f"Features nach Encoding: {X_train_full.shape[1]}")

# Fehlende Werte behandeln
imputer = SimpleImputer(strategy='median')
X_train_full = pd.DataFrame(
    imputer.fit_transform(X_train_full),
    columns=X_train_full.columns
)
X_test = pd.DataFrame(
    imputer.transform(X_test),
    columns=X_test.columns
)

# Train/Validation Split
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=42
)

print(f"\nTrain Set: {X_train.shape}")
print(f"Validation Set: {X_val.shape}")
print(f"Test Set: {X_test.shape}")

# Skalierung NUR für Features (NICHT für Target!)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("\n" + "=" * 60)
print("🔥 BUG FIXES ANGEWENDET:")
print("=" * 60)
print("1. ✅ Log-Transform VOR Feature Engineering")
print("2. ✅ price_per_room entfernt (Test-Daten-Inkonsistenz)")
print("3. ✅ Target NUR log-transformiert (kein StandardScaler!)")
print("=" * 60)
print("⚠️  WICHTIG: Target ist LOG-transformiert! Bei Predictions expm1() verwenden!")
print("=" * 60)

QUANTIL-BASIERTE AUSREISSER-ENTFERNUNG
Original Train-Größe: 16512
Untere Grenze (1% Quantil): $50,011
Max Preis im Dataset: $500,001
Preisverteilung:
  - 25% Quantil: $119,800
  - 50% Quantil: $179,850
  - 75% Quantil: $265,125
  - 99% Quantil: $500,001

Entfernte Ausreißer: 166 (1.01%)
Neue Train-Größe: 16346
Neuer Max Preis: $500,001
✅ Alle teuren Häuser bleiben erhalten (kein Ceiling!)

LOG-TRANSFORM (VOR FEATURE ENGINEERING!)
✅ Log-Transform angewendet BEVOR Feature Engineering
   Original Max: $500,001
   Log Max: 13.12

FEATURE ENGINEERING
✅ 4 neue Features erstellt:
  - rooms_per_household
  - bedrooms_ratio
  - population_density
  - income_per_bedroom

❌ price_per_room ENTFERNT (Test-Daten haben keinen Target!)

Categorical Features: ['ocean_proximity']
Numerical Features: 12
Features nach Encoding: 16

Train Set: (13076, 16)
Validation Set: (3270, 16)
Test Set: (4128, 16)

🔥 BUG FIXES ANGEWENDET:
1. ✅ Log-Transform VOR Feature Engineering
2. ✅ price_per_room entfernt (Test-D

In [4]:
print("\n" + "=" * 50)
print("SCHRITT 4: PyTorch DataLoaders erstellen")
print("=" * 50)

# Zu Tensoren konvertieren (Target ist bereits log-transformiert!)
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.FloatTensor(y_train).to(device)  # Direkt ohne StandardScaler!
X_val_tensor = torch.FloatTensor(X_val_scaled).to(device)
y_val_tensor = torch.FloatTensor(y_val).to(device)  # Direkt ohne StandardScaler!
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)

print("✅ Target NICHT mit StandardScaler skaliert (nur log-transform!)")

# DataLoaders
batch_size = 64
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

print(f"Batch Size: {batch_size}")
print(f"Training Batches: {len(train_loader)}")
print(f"Validation Batches: {len(val_loader)}")


SCHRITT 4: PyTorch DataLoaders erstellen
✅ Target NICHT mit StandardScaler skaliert (nur log-transform!)
Batch Size: 64
Training Batches: 205
Validation Batches: 52


In [5]:

print("\n" + "=" * 50)
print("SCHRITT 5: Neural Network definieren")
print("=" * 50)

class HousingPriceNN(nn.Module):
    def __init__(self, input_size, hidden_sizes=[256, 128, 64], dropout_rate=0.3):
        super(HousingPriceNN, self).__init__()
        
        layers = []
        prev_size = input_size
        
        # Hidden Layers mit BatchNorm und Dropout
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(prev_size, hidden_size))
            layers.append(nn.BatchNorm1d(hidden_size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))
            prev_size = hidden_size
        
        # Output Layer
        layers.append(nn.Linear(prev_size, 1))
        
        self.network = nn.Sequential(*layers)
        
        # Weight Initialization
        self._initialize_weights()
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        return self.network(x).squeeze()

# Model erstellen - REDUZIERTE REGULARISIERUNG!
input_size = X_train_scaled.shape[1]
model = HousingPriceNN(
    input_size=input_size,
    hidden_sizes=[512, 256, 128, 64, 32],
    dropout_rate=0.2  # 🔥 FIX: Reduziert von 0.5 auf 0.2 (zu starke Reg!)
).to(device)

print("✅ Tieferes Netzwerk: [512, 256, 128, 64, 32]")
print("🔥 Dropout Rate: 0.2 (REDUZIERT von 0.5 → NN darf lernen!)")
print(model)
print(f"\nAnzahl Parameter: {sum(p.numel() for p in model.parameters()):,}")



SCHRITT 5: Neural Network definieren
✅ Tieferes Netzwerk: [512, 256, 128, 64, 32]
🔥 Dropout Rate: 0.2 (REDUZIERT von 0.5 → NN darf lernen!)
HousingPriceNN(
  (network): Sequential(
    (0): Linear(in_features=16, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=512, out_features=256, bias=True)
    (5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): Linear(in_features=256, out_features=128, bias=True)
    (9): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Dropout(p=0.2, inplace=False)
    (12): Linear(in_features=128, out_features=64, bias=True)
    (13): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): Dropout(p=0.2, in

In [6]:
print("\n" + "=" * 50)
print("SCHRITT 6: Training Setup")
print("=" * 50)

# ============================================
# 🔥 FIX: WEIGHTED MSE LOSS (RICHTIG KONFIGURIERT!)
# ============================================
class WeightedMSELoss(nn.Module):
    """
    Custom Loss: MSE Loss mit STARKER Gewichtung für teure Häuser
    Threshold KORREKT gesetzt für teure Häuser!
    """
    def __init__(self, expensive_threshold=12.9, weight_multiplier=10.0):
        super().__init__()
        # log1p($400k) ≈ 12.90 → Nur TOP 5-10% der teuersten Häuser!
        self.expensive_threshold = expensive_threshold
        self.weight_multiplier = weight_multiplier
        self.mse = nn.MSELoss(reduction='none')
    
    def forward(self, predictions, targets):
        # Berechne MSE für jeden Sample
        losses = self.mse(predictions, targets)
        
        # Gewichte: SEHR teure Häuser (> $400k) bekommen 10x Gewicht!
        weights = torch.where(
            targets > self.expensive_threshold,
            torch.tensor(self.weight_multiplier, device=targets.device),
            torch.tensor(1.0, device=targets.device)
        )
        
        # Gewichteter Loss
        weighted_loss = (weights * losses).mean()
        return weighted_loss

criterion = WeightedMSELoss(expensive_threshold=12.9, weight_multiplier=10.0)
print("🔥 Weighted MSE Loss FIX:")
print(f"   - Threshold: 12.9 (≈ $400k) - NUR die teuersten Häuser!")
print(f"   - Gewicht: 10.0x (statt 3.0x) - VIEL STÄRKER!")
print(f"   - MSE statt Huber → Große Fehler werden STÄRKER bestraft!")
print(f"   → Nur TOP 5-10% der teuersten Häuser bekommen Extra-Gewicht")

# 🔥 Regularisierung REDUZIEREN (war zu stark!)
optimizer = optim.AdamW(
    model.parameters(), 
    lr=0.001,  # Etwas höher für besseres Lernen
    weight_decay=0.005  # VIEL schwächer (vorher 0.05!)
)
print("\n✅ Learning Rate: 0.001 (erhöht)")
print("✅ Weight Decay: 0.005 (STARK reduziert von 0.05!)")

# Learning Rate Scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=10
)

# Early Stopping
class EarlyStopping:
    def __init__(self, patience=20, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.best_model = None
    
    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_model = model.state_dict().copy()
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.best_model = model.state_dict().copy()
            self.counter = 0

early_stopping = EarlyStopping(patience=25, min_delta=0.0001)
print("\n" + "=" * 60)
print("🔥 CEILING EFFECT FIXES V2:")
print("=" * 60)
print("1. ✅ Weighted MSE: Threshold 12.9 ($400k), Weight 10x")
print("2. ✅ MSE statt Huber (bestraft große Fehler STÄRKER!)")
print("3. ✅ Weight Decay 0.05 → 0.005 (10x schwächer!)")
print("4. ✅ Nur TOP 5-10% teuerste Häuser extra gewichtet")
print("=" * 60)
print("→ NN darf jetzt hohe Werte vorhersagen!")
print("→ Fehler bei teuren Häusern werden STARK bestraft!")
print("=" * 60)



SCHRITT 6: Training Setup
🔥 Weighted MSE Loss FIX:
   - Threshold: 12.9 (≈ $400k) - NUR die teuersten Häuser!
   - Gewicht: 10.0x (statt 3.0x) - VIEL STÄRKER!
   - MSE statt Huber → Große Fehler werden STÄRKER bestraft!
   → Nur TOP 5-10% der teuersten Häuser bekommen Extra-Gewicht

✅ Learning Rate: 0.001 (erhöht)
✅ Weight Decay: 0.005 (STARK reduziert von 0.05!)

🔥 CEILING EFFECT FIXES V2:
1. ✅ Weighted MSE: Threshold 12.9 ($400k), Weight 10x
2. ✅ MSE statt Huber (bestraft große Fehler STÄRKER!)
3. ✅ Weight Decay 0.05 → 0.005 (10x schwächer!)
4. ✅ Nur TOP 5-10% teuerste Häuser extra gewichtet
→ NN darf jetzt hohe Werte vorhersagen!
→ Fehler bei teuren Häusern werden STARK bestraft!


In [7]:
print("\n" + "=" * 50)
print("SCHRITT 7: Training starten")
print("=" * 50)

def train_epoch(model, loader, criterion, optimizer):
    model.train()
    total_loss = 0
    for X_batch, y_batch in loader:
        optimizer.zero_grad()
        predictions = model(X_batch)
        loss = criterion(predictions, y_batch)
        loss.backward()
        
        # Gradient Clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def validate(model, loader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in loader:
            predictions = model(X_batch)
            loss = criterion(predictions, y_batch)
            total_loss += loss.item()
    return total_loss / len(loader)

# Training Loop
num_epochs = 200
train_losses = []
val_losses = []

print(f"Training für {num_epochs} Epochen...")
print(f"Device: {device}\n")

for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_loader, criterion, optimizer)
    val_loss = validate(model, val_loader, criterion)
    
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    
    # Learning Rate anpassen
    scheduler.step(val_loss)
    
    # Early Stopping prüfen
    early_stopping(val_loss, model)
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}] - "
              f"Train Loss: {train_loss:.6f} - "
              f"Val Loss: {val_loss:.6f} - "
              f"LR: {optimizer.param_groups[0]['lr']:.6f}")
    
    if early_stopping.early_stop:
        print(f"\nEarly Stopping bei Epoch {epoch+1}")
        break

# Bestes Model laden
model.load_state_dict(early_stopping.best_model)
print("\nBestes Model geladen!")


SCHRITT 7: Training starten
Training für 200 Epochen...
Device: mps



KeyboardInterrupt: 

In [ ]:
print("\n" + "=" * 50)
print("SCHRITT 8: Model Evaluation")
print("=" * 50)

# Vorhersagen auf Validation Set
model.eval()
with torch.no_grad():
    val_predictions_log = model(X_val_tensor).cpu().numpy()

# 🔥 FIX: Nur expm1() verwenden (kein target_scaler mehr!)
val_predictions = np.expm1(val_predictions_log)
y_val_original = np.expm1(y_val)  # y_val ist bereits log-transformiert

# Metriken berechnen
mse = mean_squared_error(y_val_original, val_predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_val_original, val_predictions)
r2 = r2_score(y_val_original, val_predictions)

print(f"\nValidation Metriken (Original-Skala):")
print(f"RMSE: ${rmse:,.2f}")
print(f"MAE: ${mae:,.2f}")
print(f"R² Score: {r2:.4f}")
print(f"MAPE: {np.mean(np.abs((y_val_original - val_predictions) / y_val_original)) * 100:.2f}%")
print("\n✅ Inverse Transform: expm1() only (kein StandardScaler!)")


SCHRITT 8: Model Evaluation

Validation Metriken (Original-Skala):
RMSE: $4,681,091.44
MAE: $124,426.30
R² Score: -1653.7688
MAPE: 38.28%

✅ Inverse Transform: expm1() only (kein StandardScaler!)
